### Mesh processing for SimuCell3D

This notebook outlines how to generate a geometry for the SimuCell3D.

In [28]:
import os
import sys
import numpy as np
from MeshPrep import convert_filtered_meshes
from MeshPrep import mesh_process_clean, string_to_array

#### Cleaning the labels

The labels generated from manual curation should be processesed before passing through the geometry generation. Strongly recommended step. Generates meshes which may be used for manual cell patch selection using paraview.

In [ ]:
# root = '../outputs/outputs_v3/output_lung_new_sample_b_curated_segmentation_central_crop_relabel_seq_s_10_e_6_d_8/'

In [ ]:
# voxel_resolution = np.array([0.1625, 0.1625, 0.25])
# label_path = os.path.join(root, 'processed_labels.tif')

In [ ]:
# from VoxelProcessing import full_label_processing

# output_folder='path/folder/to/save/processed/labels'
# cell_info = full_label_processing(labeled_img=label_path, voxel_resolution=voxel_resolution, output_folder=output_folder, smoothing_iterations=10)
# label_path = os.path.join(output_folder, "processed_labels.npy")
# filtered_cell_list = cell_info[2]

#### Select labels for Simulations

To obtain labels for simulation one may use napari and select cells manually, or use meshes, and paraview. View how to use paraview [here](Tutorials/LabelSelection.md).

You may want to isolate the cells that do not touch the border instead. To isolate these use the following script and use Paraview's extract functionality as outlined.

In [ ]:
from FilterMeshes import get_valid_cell_ids

root = '../outputs/outputs_v3/output_lung_new_sample_b_curated_segmentation_central_crop_relabel_seq_s_10_e_6_d_8/'

filtered_cell_list = get_valid_cell_ids(os.path.join(root, 'cell_stats/stats_dataset_lung_bronchiole.csv'))

In [ ]:
# The directory from which mesh files are loaded in stl format
source_mesh_path = os.path.join(root, 'cell_meshes')

# The directory in which filtered meshes will be saved in vtk format
dest_mesh_path = '../../Meshes_for_Simulation/examples/cell_clump_bronchiole' 

convert_filtered_meshes(source_mesh_path, dest_mesh_path, filtered_cell_list)

Now you can open the `.vtk` mesh files in paraview and select a clump of cells for simulation.

#### Mesh Cleaning for SimuCell3D

Produce meshes for SimuCell3D simulation framework. Ensure that the path to labels is of cleaned, processed labels. 

In [2]:
root = '../outputs/outputs_v4/output_intestine_sample2_b_curated_segmentation_relabel_seq_s_10_e_6_d_8/'
voxel_resolution = np.array([0.325, 0.325, 0.25])
label_path = os.path.join(root, 'processed_labels.tif')

label_list = string_to_array("128 130 138 139 147 150 163 167 169 171 180 185 187 193 210 228")
output_dir = '../../Meshes_for_Simulation/examples/cell_clump_intestine/cell_clumps/clean_clump_16_cells/clean_meshes/'

# Call the mesh_process_clean function
mesh_process_clean(
    label_path=label_path, 
    output_dir=output_dir, 
    label_list=label_list, 
    voxel_resolution=voxel_resolution, 
    scale_factor=1e-6, 
    min_edge_length=0.5,
    make_shell=True
)

-------------------------------------------
Creating meshes from labeled img...


Converting labels to meshes: 100%|██████████| 16/16 [03:30<00:00, 13.13s/it]


-------------------------------------------
First mesh cleaning...


Cleaning non-mainfold meshes:   0%|          | 0/18 [00:00<?, ?it/s]

Removed 20 non-manifold faces
INFO- Loaded 12263 vertices and 24512 faces.

100% done 
INFO- ********* ITERATION 0 *********
INFO- Removing degeneracies...
INFO- Removing self-intersections...

98 % done   
INFO- No intersections detected.

0% done 


...

Removed 0 non-manifold faces
INFO- Loaded 7621 vertices and 15238 faces.

0% done 
INFO- ********* ITERATION 0 *********
INFO- Removing degeneracies...
INFO- Removing self-intersections...

99 % done   
INFO- No intersections detected.

0% done 
-------------------------------------------
Getting `.vtk` files...


Converting files to .vtk: 100%|██████████| 17/17 [00:02<00:00,  5.67it/s]


-------------------------------------------
Preparing file for simulation...


Merging .vtk files: 100%|██████████| 17/17 [00:00<00:00, 27.01it/s]


'../../Meshes_for_Simulation/examples/cell_clump_intestine/cell_clumps/clean_clump_16_cells/clean_meshes/vtk_files/merged.vtk'

NOTE: To create a shell mesh that fits  really well the single cell meshes you may need to resort to the simulation pipeline.

In that case you need to do the following:
- In the source code (./include/mesh/cell_types), set for ECM cells `is_static=False` in the constructors, while set it to `True` for the epithelial cells.
- Set surface tension values for ECM in at the middle of the "feasible range" experimented for epithelial cells (e.g., 1e-3 could work).
- Run few simulation iterations untile the result is satisfactory. Take the resulting vtk file as the input for following simulation runs.

## Test 'creation of outer shell mesh'

In [1]:
import napari
import numpy as np
from scipy.spatial import Delaunay
import trimesh
import open3d as o3d
from trimesh.proximity import signed_distance
# from MeshPrep import create_outer_shell_mesh
from trimesh.creation import icosphere
from typing import List

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def create_sample_spheres(
        n: int,
        radiuses: List[float]
) -> List[trimesh.Trimesh]:
    """
    Create n spherical meshes displaced one next to the other and with centers aligned.

    Parameters:
    -----------
    n: (int)
        The number of spheres to create
    radiuses: (List[int])
        The radius of the spheres

    Returns:
    --------
    spheres: (List[trimesh.Trimesh])
        A list of meshes of spheres
    """
    spheres = []
    displacement = np.array([0.0, 0.0, 0.0])
    for i in range(n):
        curr_sphere = icosphere(radius=radiuses[i])
        curr_sphere.vertices += displacement
        displacement[0] += 2 * radiuses[i]
        spheres.append(curr_sphere)

    return spheres

#### Example with spheres

In [4]:
spheres = create_sample_spheres(3, [1, 1, 1])

In [5]:
neighbors_lst = [[1], [0, 2], [1]]

In [6]:
# Calculate the length of each edge
edges = spheres[0].edges
edge_lengths = spheres[0].vertices[edges[:, 0]] - spheres[0].vertices[edges[:, 1]]
edge_lengths = trimesh.util.row_norm(edge_lengths)

# Get the shortest edge length
shortest_edge_length = min(edge_lengths)
print(shortest_edge_length)

0.13828317354716763


In [46]:
outer_shell_mesh, outer_shell_cloud, outer_shell_pts = create_outer_shell_mesh(
    spheres,
    neighbors_lst,
    shortest_edge_length / 3,
    shortest_edge_length / 3
)

---------------------------------------------------
Current mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, curr neighbors: [1]
Original num of vertices: (642, 3)
Max distance: 2.0, mean: 1.168977044593609, std: 0.5526613728851824
Remaining num of vertices: (635, 3)
---------------------------------------------------
Current mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, curr neighbors: [0, 2]
Original num of vertices: (642, 3)
Max distance: 2.0, mean: 1.168977044593609, std: 0.5526613728851824
Remaining num of vertices: (635, 3)
Max distance: 1.98105595120421, mean: 1.15987007327802, std: 0.5488123902392724
Remaining num of vertices: (628, 3)
---------------------------------------------------
Current mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, curr neighbors: [1]
Original num of vertices: (642, 3)
Max distance: 2.0, mean: 1.168977044593609, std: 0.5526613728851824
Remaining num of vertices: (635, 3)


In [47]:
viewer = napari.Viewer()
for sphere in spheres:
    viewer.add_surface((sphere.vertices, sphere.faces))
viewer.add_surface((outer_shell_mesh.vertices, outer_shell_mesh.faces), opacity=0.5, blending="additive")
viewer.add_points(outer_shell_pts, face_color="red", size=0.1)

<Points layer 'outer_shell_pts' at 0x1bb4ebe2e60>

In [53]:
o3d.io.write_point_cloud(r"../../../shell_pcd_2.ply", outer_shell_cloud)

True

#### Example with cell meshes

In [54]:
mesh1 = trimesh.load_mesh(r"N:\Users\Federico_Carrara\Meshes_for_Simulation\examples\cell_clump_intestine\cell_clumps\clean_clump_16_cells\clean_meshes\cell_210.stl")
mesh2 = trimesh.load_mesh(r"N:\Users\Federico_Carrara\Meshes_for_Simulation\examples\cell_clump_intestine\cell_clumps\clean_clump_16_cells\clean_meshes\cell_228.stl")

In [57]:
mesh_lst = [mesh1, mesh2]
neighbors_lst = [[1], [0]]

# Calculate the length of each edge
edges = mesh1.edges
edge_lengths = mesh1.vertices[edges[:, 0]] - mesh1.vertices[edges[:, 1]]
edge_lengths = trimesh.util.row_norm(edge_lengths)

# Get the shortest edge length
shortest_edge_length = min(edge_lengths)
print(shortest_edge_length)

0.09302981215157372


In [58]:
outer_shell_mesh, outer_shell_cloud, outer_shell_pts = create_outer_shell_mesh(
    mesh_lst,
    neighbors_lst,
    shortest_edge_length / 3,
    shortest_edge_length / 3
)

---------------------------------------------------
Current mesh: <trimesh.Trimesh(vertices.shape=(17361, 3), faces.shape=(34722, 3))>, curr neighbors: [1]
Original num of vertices: (17361, 3)


### Test new OOP approach

In [59]:
from OuterShell import ExtendedTrimesh, OuterShell

In [60]:
spheres = create_sample_spheres(3, [1, 1, 1])
neighbors_lst = [[1], [0, 2], [1]]

In [61]:
# Calculate the length of each edge
edges = spheres[0].edges
edge_lengths = spheres[0].vertices[edges[:, 0]] - spheres[0].vertices[edges[:, 1]]
edge_lengths = trimesh.util.row_norm(edge_lengths)

# Get the shortest edge length
shortest_edge_length = min(edge_lengths)

In [62]:
extended_spheres = []
for i, sphere in enumerate(spheres):
    extended_spheres.append(ExtendedTrimesh(neighbors=neighbors_lst[i], vertices=sphere.vertices, faces=sphere.faces))

In [63]:
outer_shell = OuterShell(extended_spheres, neighbors_lst=neighbors_lst, min_edge_length=shortest_edge_length)

In [64]:
outer_shell.get_shell_point_cloud(1)

TypeError: Partition index must be integer

In [21]:
outer_shell._meshes[2]

<trimesh.ExtendedTrimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>

In [32]:
outer_shell.interpolate_gaps('spline')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices